# Default settings

In [1]:
# config defaults
default_config = {
    "SKULLSTRIP": "1",
    "DVARS": "1",
    "SLICETIME": "1",
    "MOTCOR": "1",
    "NORM": "1",
    "SMOOTH": "6", # gaussian smoothing kernel size in mm
    "MOTREG": "1",
    "DVARS": "1",
    "FD": "1",
    "TEMPLATE": "/usr/local/fsl/data/standard/MNI152_T1_2mm_brain.nii.gz",
    "OUTLIER": "UNION" # union, intersect, dvars, fd, none
}

# Support functions

In [2]:
def create_dirstruct(output):
    import pathlib 
    import os

    pathlib.Path(os.path.join(output, "func"     )).mkdir(parents=True, exist_ok=True)
    pathlib.Path(os.path.join(output, "anat"     )).mkdir(parents=True, exist_ok=True)
    pathlib.Path(os.path.join(output, "spat_norm")).mkdir(parents=True, exist_ok=True)
    pathlib.Path(os.path.join(output, "motion"   )).mkdir(parents=True, exist_ok=True)

    return

In [3]:
def check_exist(filepath, type="any"):
    if type=='f':
        type='file'
    elif type=='d':
        type='dir'

    if type=='file':
        from os.path import isfile as exists
    elif type=="dir":
        from os.path import isdir as exists
    elif type=="any":
        from os.path import exists
    else:
        raise Exception("type '" + type + "' is invalid.")
    
    return(exists(filepath))

In [4]:
def parse_line(text, keywords, split="="):
    key, val = text.split("=")
    if key not in keywords:
        raise Exception("Input doesn't contain any of the specified key words")
    return(key, val)

In [5]:
def strip_chars(text, first, last):
    text = "{}".format(text[1:] if text.startswith(first) else text)
    text = "{}".format(text[:-1] if text.endswith(last) else text)
    return(text)

In [6]:
def parse_input(text, keywords, split="=", first="[", last="]"):
    key, val = parse_line(text, keywords, split)
    val = strip_chars(val, "[", "]")
    return(key, val)

In [7]:
def check_defaults(config_dict, defaults):
    default_keys=[key for key in defaults.keys()]

    restore_defaults=[key not in config_dict.keys() for key in default_keys]
    restore_defaults=list(filter(lambda x: restore_defaults[x], range(len(restore_defaults))))
    restore_defaults=[default_keys[i] for i in restore_defaults]

    for key in restore_defaults:
        config_dict[key] = defaults[key]

    return(config_dict)


In [8]:
def interpret_input_line(line, keywords):
    input_dict = {}
    items = line.split(",")
    
    for item in items:
        item=item.strip()
        
        try:
            key, val = parse_input(item, keywords) # keywords must be enterest as a list
            input_dict[key] = val
        except:
            print("Error:", item)

    # check if all the files exist 
    if not check_exist(input_dict["FUNC"], "file"):
        raise Exception(input_dict["FUNC"] + " is not a valid filepath.")
    if not check_exist(input_dict["ANAT"], "file"):
        raise Exception(input_dict["ANAT"] + " is not a valid filepath.")
       
    return(input_dict)

In [9]:
# interpret input
def interpret_input(filepath):
    import pandas as pd

    input=open(filepath, "r")
    lines=input.readlines()

    keywords = ["FUNC", "ANAT", "OUTPUT"]
    input_df = pd.DataFrame(index=range(len(lines)), columns=keywords)

    for line, ind in zip(lines, range(len(lines))):
        line=line.strip()
        input_dict=interpret_input_line(line, keywords)

        input_df.loc[ind,"FUNC"  ] = input_dict["FUNC"]
        input_df.loc[ind,"ANAT"  ] = input_dict["ANAT"]
        input_df.loc[ind,"OUTPUT"] = input_dict["OUTPUT"]

    return(input_df)
    

In [10]:
# interpret config
def interpret_config(filepath):
    config=open(filepath, "r")
    lines=config.readlines()

    keywords=["SKULLSTRIP","DVARS", "FD", "SLICETIME","MOTCOR","NORM", "SMOOTH", "MOTREG", "OUTLIER", "TEMPLATE"]

    config_dict = {}

    for line in lines:
        line=line.strip()
        try:
            key, val = parse_input(line, keywords) # keywords must be enterest as a list
            config_dict[key] = val
        except:
            print("Error:", line)

    config_dict = check_defaults(config_dict, default_config)
    return(config_dict)

In [11]:
def afni2nifti(filepath, rm_orig=True):
    import os
    
    nii_filepath=os.path.join(os.path.dirname(filepath), rm_ext(filepath))+".nii"
    os.system("3dAFNItoNIFTI -prefix {} {}".format(nii_filepath, filepath+"*.HEAD"))
    if rm_orig:
        os.system("rm {}*.HEAD".format(filepath))
        os.system("rm {}*.BRIK".format(filepath))
    return(nii_filepath)

In [12]:
def getTR(filepath):
    import subprocess
    
    tr=subprocess.check_output("fslval {} pixdim4".format(filepath), shell=True)
    tr=tr.decode("utf-8")
    tr=tr.rstrip()
    return(tr) 

In [13]:
def rm_ext(filename):
    from os.path import splitext
    from os.path import basename
    
    filename=basename(filename)
    newpath=splitext(filename)[0]
    
    if "." in newpath:
        newpath=rm_ext(newpath)
    return(newpath)

In [14]:
def get_ext(filename):
    from os.path import splitext
    from os.path import basename

    filename=basename(filename)
    newname, ext=splitext(filename)
    
    if "." in newname:
        ext = get_ext(newname) + ext
    return(ext)

In [15]:
def gauss_mm2sigma(mm):
    sigma=mm/2.35482004503
    return(sigma)


In [16]:
def which(list, x=True):
    coords=[]
    iter=0
    
    for elem in list:
        if elem == x:
            coords.append(iter)
        iter+=1
    return(coords)

In [17]:
def gzip_file(filename):
    import gzip
    import shutil
    
    with open(filename, 'rb') as f_in:
        with gzip.open(filename+'.gz', 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
    return(filename+'.gz')

# Preprocessing Functions

In [18]:
def brainroi(img, out_dir):
    import os

    roi_img = os.path.join(out_dir, "roi_"+os.path.basename(img))
    os.system("robustfov -i {} -r {}".format(img, roi_img))

    return(roi_img)


In [19]:
def skullstrip(img, out_dir):
    import os

    out_dir  = os.path.join(out_dir, "anat")
    out_file = "brain_" + rm_ext(img)
    skullstr = os.path.join(out_dir, out_file)

    print("SKULLSTRIPPING - skipped")

    # os.system("bet {} {} -R".format(img, skullstr))
    return(skullstr)

In [20]:
def slicetime(img, out_dir):
    import os

    tr=getTR(img)
    tshift_path=os.path.join(out_dir, "func", "t_" + rm_ext(img))
    
    print("SLICETIME CORRECTION - skipped")
    # os.system("3dTshift -TR {}s -prefix {} {}".format(tr, tshift_path, img))

    tshift_path=afni2nifti(tshift_path)
    return(tshift_path)

In [21]:
def motcor(img, out_dir):
    import os

    prefix="m"

    motcor_path=os.path.join(out_dir, "func", "m_"+os.path.basename(img))
    _1dfile_path=os.path.join(out_dir, "motion", "1d_"+os.path.splitext(os.path.basename(img))[0]+".1D")

    print("MOTION CORRECTION - skipped")
    # os.system("3dvolreg -base 0 -prefix {} -1Dfile {} {}".format(motcor_path, _1dfile_path, img))
    # afni2nifti(motcor_path)

    return(motcor_path, _1dfile_path)

In [22]:
# new spatial normalization
def spatnorm(f_img, a_img, template, out_dir):
    print("SPATIAL NORMALIZATION - skipped")
    import os

    # lin warp func to struct
    print("       + Linear-warping functional to structural...")
    l_func_omat=os.path.join(out_dir, "spat_norm", "func2str.mat")
    # os.system("flirt -ref {} -in {} -omat {} -dof 6".format(a_img, f_img, l_func_omat))
    
    # lin warp struct to template
    print("       + Linear-warping structural to standard template...")
    l_anat_img =os.path.join(out_dir, "anat", "l_" + os.path.basename(a_img))
    l_anat_omat=os.path.join(out_dir, "spat_norm", "aff_str2std.mat")
    # os.system("flirt -ref {} -in {} -omat {} -out {}".format(template, a_img, l_anat_omat, l_anat_img))
    
    # non-lin warp struct to template
    print("       + Non-linear-warping structural to standard template...")
    nl_anat_img  =os.path.join(out_dir, "anat", "n" + os.path.basename(l_anat_img))
    cout_anat_img=os.path.join(out_dir, "anat", "cout_" + os.path.basename(nl_anat_img))
    # os.system("fnirt --ref={} --in={} --aff={} --iout={} --cout={} --subsamp=2,2,2,1".format(template, a_img, l_anat_omat, nl_anat_img,cout_anat_img))
    
    # make binary mask from non-lin warped image
    print("       + Creating binary mask from non-linearly warped image...")
    bin_nl_anat_img=os.path.join(out_dir, "anat", "bin_" + os.path.basename(nl_anat_img))
    # os.system("fslmaths {} -bin {}".format(nl_anat_img, bin_nl_anat_img))
    
    # apply std warp to func data
    print("       + Applying standardized warp to functional data...")
    nl_func_img=os.path.join(out_dir, "func", "nl_"+os.path.basename(f_img))
    # os.system("applywarp --ref={} --in={} --out={} --warp={} --premat={}".format(template, f_img, nl_func_img, cout_anat_img, l_func_omat))

    # remove unwanted files

    return(nl_anat_img, nl_func_img) # anat, func

In [23]:
def spat_smooth(img, mm, out_dir):
    import os

    print("SMOOTHING FUNCTIONAL DATA - skipped")
    
    sigma=gauss_mm2sigma(mm)
    s_img=os.path.join(out_dir, "s_"+os.path.basename(img))
    
    # os.system("fslmaths {} -kernel gauss {} -fmean {}".format(img, sigma, s_img))
    return(s_img)

In [24]:
def motreg(img, _1d, out_dir):
	import os

	print("MOTION REGRESSION - skipped")

	error_img =os.path.join(out_dir, "error_"  + rm_ext(img) + ".nii")
	motreg_img=os.path.join(out_dir, "motreg_" + rm_ext(img) + ".nii")

	# os.system(
	# "3dDeconvolve \
	# 	-input {} \
	# 	-num_stimts 6 \
	# 	-stim_file 1 {}'[0]' -stim_base 1 -stim_label 1 roll  \
	# 	-stim_file 2 {}'[1]' -stim_base 2 -stim_label 2 pitch \
	# 	-stim_file 3 {}'[2]' -stim_base 3 -stim_label 3 yaw   \
	# 	-stim_file 4 {}'[3]' -stim_base 4 -stim_label 4 dS    \
	# 	-stim_file 5 {}'[4]' -stim_base 5 -stim_label 5 dL    \
	# 	-stim_file 6 {}'[5]' -stim_base 6 -stim_label 6 dP    \
	# 	-fitts {} \
	# 	-errts {}".format(img, _1d, _1d, _1d, _1d, _1d, _1d, motreg_img, error_img))

	# error_img =afni2nifti(error_img)
	# motreg_img=afni2nifti(motreg_img)

	return(motreg_img, error_img)

In [25]:
def deg2mm(_1d_file):
    import os
    import pandas as pd
    from math import pi

    rad = 50 # radius of head in mm
    circum = 2 * pi * rad

    MPEs = pd.read_csv(_1d_file, sep=r"\s+", header=None)
    MPEs.columns = ['roll', 'pitch', 'yaw', 'dS', 'dL', 'dP']
    MPEs.loc[:,['dS', 'dL', 'dP']] = circum * (MPEs.loc[:,['dS', 'dL', 'dP']] / 360)

    out_file = os.path.join(os.path.dirname(_1d_file), "mm_"+os.path.basename(_1d_file))
    
    MPEs.to_csv(out_file, sep="\t")

    return(out_file)

In [26]:
def meantsBOLD(func_img, mot_dir):
    import os
    import pandas as pd

    dvars_txt = os.path.join(mot_dir, "dvars.txt")
    dvars_png = os.path.join(mot_dir, "dvars.png")

    out_compound = os.path.join(mot_dir, "dvars_outliers.txt")

    os.system("fsl_motion_outliers -i {} -o {} -s {} -p {} --nomoco".format(func_img, out_compound, dvars_txt, dvars_png))

    # append extra 0
    # dvars_out=open(out_compound, "a")
    # dvars_out.write("0")
    # dvars_out.close()

    dvars_out=pd.read_csv(out_compound, delim_whitespace=True, header=0)
    dvars_out=dvars_out.sum(axis=1)
    dvars_out=dvars_out.astype("int")

    dvars_out.to_csv(out_compound, sep="\t", index=False)

    return(out_compound)

In [27]:
def fd_out(path, voxel_size):
    import os
    import pandas as pd

    fd_data    = pd.read_csv(path, sep="\t", index_col = 0, header=0)
    voxel_size = min([float(i) for i in voxel_size.split("\t")])

    out_file = os.path.join(os.path.dirname(path), "fd_outliers.txt")

    out_ind = (fd_data > voxel_size).sum(axis=1)
    out_ind = out_ind.astype('int')
    out_ind.to_csv(out_file, sep="\t", index=False, header=False)

    return(out_file)

In [28]:
def mk_outliers(dvars, fd, out_dir, method="union"):
    import os
    import pandas as pd

    print("FLAGGING OUTLIERS")

    fd_mat    = pd.read_csv(fd   , delimiter="\t", header=None)
    dvars_mat = pd.read_csv(dvars, delimiter="\t", header=None)

    if method.lower()=="union":
        outliers_mat = (fd_mat + dvars_mat).astype("bool")
        outliers_mat = outliers_mat.astype("int")
    elif method.lower()=="intersect":
        outliers_mat = dvars_mat[0]*fd_mat[0]
        outliers_mat = outliers_mat.dropna()
    elif method.lower()=="fd":
        outliers_mat = fd_mat
    elif method.lower()=="dvars":
        outliers_mat = dvars_mat
    
    outliers_mat.to_csv(os.path.join(out_dir, "outliers.txt"), sep="\t", index=False, header=False)

    return(os.path.join(out_dir, "outliers.txt")


SyntaxError: unexpected EOF while parsing (<ipython-input-28-be6b1eca36da>, line 23)

In [29]:
def interp_time(img, out, int_ind, return_full=False):
    import numpy as np

    if len(img.shape)!=4:
        raise Exception("Nifti image must have 4 dimensions.")
    
    # make blank image
    int_dim = list(img.shape[0:3])
    int_dim.append(int_ind[1]-int_ind[0]-1)
    int_img = np.empty(int_dim)

    for row in range(img.shape[0]):
        for col in range(img.shape[1]):
            for slice in range(img.shape[2]):
                int_val = interp_pts(img[row, col, slice, int_ind[0]], img[row, col, slice, int_ind[1]], int_ind[1]-int_ind[0]-1)
                int_img[row, col, slice, :] = int_val

    if return_full:
        img[:,:,:,out] = int_img
        return(img)
    else:
        return(int_img)


In [30]:
def interp_pts(start, end, nvals, round=True):
    
    step_len=(end-start)/(nvals+1)
    
    interp_set=[start+(step_len*i) for i in range(1,nvals+1)]
    if round:
        interp_set=round_dec(interp_set, [start, end])

    return(interp_set)

In [31]:
def round_dec(vals, ref):
    import decimal

    n_places=max([n_dec(i) for i in ref])
    rnd_vals=[round(i, n_places) for i in vals]

    return(rnd_vals)

In [32]:
def n_dec(val):
    import decimal

    n_places=abs(decimal.Decimal(str(val)).as_tuple().exponent)
    return(n_places)

In [33]:
def surrounding_timepoints(t_range, ind):

    import numpy as np
    
    sub_ind = np.empty((0,2), dtype=int, order='C')
    
    for ii in ind:
        t_start = ii-1
        t_end   = ii+1
        
        while t_start in ind:
            t_start -= 1
        while t_end in ind:
            t_end += 1
        
        if t_start < min(t_range):
            t_start = "NaN"
        if t_end < min(t_range):
            t_end = "NaN"
        
        sub_ind = np.vstack([sub_ind, (t_start, t_end)])
        sub_ind = np.unique(sub_ind, axis=0)
    return(sub_ind)

In [40]:
def scrubbing(nifti, outliers, out_dir, interpolate=False):
    import os
    import nibabel as nib
    import numpy as np
    from pandas import read_csv
    from nibabel.testing import data_path
    from scipy.interpolate import RegularGridInterpolator

    print("SCRUBBING fMRI TIME SERIES")
    print(outliers)

    img = nib.load(nifti)
    mat = img.get_fdata()

    outliers=read_csv(outliers, delimiter="\t", header=None)
    outliers=outliers.values.tolist()
    outliers=which([o[0] for o in outliers], 1)

    print("       + Removing outlier timepoints...")
    mat[:,:,:,outliers]=float("NaN")

    if interpolate:
        print("       + Linearly-interpolating outliers...")
        sub_ind = surrounding_timepoints(range(mat.shape[3]), outliers)

        for oo in range(len(sub_ind)):
            tmp = interp_time(mat, outliers[oo], sub_ind[oo,:], return_full=False)
            mat[:,:,:,(sub_ind[oo,0]+1):sub_ind[oo,1]] = tmp

    new_img = nib.Nifti1Image(mat, img.affine, header=img.header)
    print("       + Saving scrubbed timeseries...")
    nib.save(new_img, os.path.join(out_dir, 'n_scrubbed_image.nii'))
    
    return(os.path.join(out_dir, 'n_scrubbed_image.nii'))


# Wrappers

In [31]:
def wrapper_lvl2(input_file, config_file):
    config=interpret_config(config_file)
    input =interpret_input(input_file)

    nrow = len(input.index)
    
    for row in range(nrow):
        wrapper_lvl1(input.loc[row,:], config)
        print("")
    return(input, config)

In [33]:
def wrapper_lvl1(input, config):
    import os
    import subprocess

    create_dirstruct(input["OUTPUT"])

    cur_func=os.path.join(input["OUTPUT"], "func", "func" + get_ext(input["FUNC"]))
    cur_anat=os.path.join(input["OUTPUT"], "anat", "MPRage" + get_ext(input["ANAT"]))

    os.system("cp {} {}".format(input["FUNC"], cur_func))
    os.system("cp {} {}".format(input["ANAT"], cur_anat))

    # cur_anat = brainroi(cur_anat, os.path.join(input["OUTPUT"], "anat"))

    anat_nobet = cur_anat

    if config["SKULLSTRIP"]=="1":
        cur_anat=skullstrip(cur_anat, input["OUTPUT"])
    if config["SLICETIME"]=="1":
        cur_func=slicetime(cur_func, input["OUTPUT"])
    if config["MOTCOR"]=="1":
        cur_func, _1dfile_path=motcor(cur_func, input["OUTPUT"])

        voxel_size=subprocess.check_output("3dinfo -adi -adj -adk {}".format(cur_func), shell=True)
        voxel_size=voxel_size.decode("utf-8")
        voxel_size=voxel_size.rstrip()

        mm_1dfile_path=deg2mm(_1dfile_path)
    if config["FD"]=="1":
        os.system("Rscript framewise_disp.R --OUT_DIR={} --MPE_MM={} --VOXEL_DIM={}".format( \
            os.path.join(input["OUTPUT"], "motion"), _1dfile_path, voxel_size))
        fd_path=os.path.join(input["OUTPUT"], "motion", 'motion_FD.csv')
        fd_outliers=fd_out(fd_path, voxel_size)
    if config["DVARS"]=="1":
        dvar_out = meantsBOLD(cur_func, os.path.join(input["OUTPUT"], "motion"))
    if config["NORM"]=="1":
        cur_anat, cur_func=spatnorm(cur_func, cur_anat, config["TEMPLATE"], input["OUTPUT"])
    if float(config["SMOOTH"]) > 0:
        cur_func=spat_smooth(cur_func, float(config["SMOOTH"]), os.path.join(input["OUTPUT"], "func"))
    if config["MOTREG"]=="1":
        cur_func, err_func=motreg(cur_func, _1dfile_path, os.path.join(input["OUTPUT"], "func"))
    if config["OUTLIER"].lower()!="none":
        outliers=mk_outliers(dvar_out, fd_outliers, os.path.join(input["OUTPUT"], "motion"), method=config["OUTLIER"].lower())
        scrubbed_nifti=scrubbing(cur_func, outliers, os.path.join(input["OUTPUT"], "func"), interpolate=True)

    return(input, config)


# TESTING AREA

In [32]:
input_filepath  = "/mnt/d/Downloads/ds000031-download/input.txt"
config_filepath = "/mnt/d/Downloads/ds000031-download/config.txt"

input, config=wrapper_lvl2(input_filepath, config_filepath)

NameError: name 'wrapper_lvl1' is not defined

In [53]:
img="/mnt/d/Downloads/ds000031-download/sub-01/ses-003/out/func/t_func.nii.nii"
out_dir="/mnt/d/Downloads/ds000031-download/sub-01/ses-003/out"
motcor(img, out_dir)

/mnt/d/Downloads/ds000031-download/sub-01/ses-003/out/func/m_t_func.nii.nii
/mnt/d/Downloads/ds000031-download/sub-01/ses-003/out/motion/1d_t_func.nii.1D
MOTION CORRECTION


('/mnt/d/Downloads/ds000031-download/sub-01/ses-003/out/func/m_t_func.nii.nii',
 '/mnt/d/Downloads/ds000031-download/sub-01/ses-003/out/motion/1d_t_func.nii.1D')